# T3. dataloader 的内部结构和基本使用

&emsp; 1 &ensp; fastNLP 中的 dataloader
 
&emsp; &emsp; 1.1 &ensp; dataloader 的基本介绍

&emsp; &emsp; 1.2 &ensp; dataloader 的函数创建

&emsp; 2 &ensp; fastNLP 中 dataloader 的延伸

&emsp; &emsp; 2.1 &ensp; collator 的概念与使用

&emsp; &emsp; 2.2 &ensp; sampler 的概念与使用

## 1. fastNLP 中的 dataloader

### 1.1 dataloader 的基本介绍

在`fastNLP 0.8`的开发中，最关键的开发目标就是**实现`fastNLP`对当前主流机器学习框架**，例如

&emsp; **较为火热的`pytorch`**，以及**国产的`paddle`和`jittor`的兼容**，扩大受众的同时，也是助力国产

本着分而治之的思想，我们可以将`fastNLP 0.8`对`pytorch`、`paddle`、`jittor`框架的兼容，划分为

&emsp; &emsp; **对数据预处理**、**批量`batch`的划分与补齐**、**模型训练**、**模型评测**，**四个部分的兼容**

&emsp; 针对数据预处理，我们已经在`tutorial-1`中介绍了`dataset`和`vocabulary`的使用

&emsp; &emsp; 而结合`tutorial-0`，我们可以发现**数据预处理环节本质上是框架无关的**

&emsp; &emsp; 因为在不同框架下，读取的原始数据格式都差异不大，彼此也很容易转换

只有涉及到张量、模型，不同框架才展现出其各自的特色：**`pytorch`中的`tensor`和`nn.Module`**

&emsp; &emsp; **在`paddle`中称为`tensor`和`nn.Layer`**，**在`jittor`中则称为`Var`和`Module`**

&emsp; &emsp; 因此，**模型训练、模型评测**，**是兼容的重难点**，我们将会在`tutorial-5`中详细介绍

&emsp; 针对批量`batch`的处理，作为`fastNLP 0.8`中框架无关部分想框架相关部分的过渡

&emsp; &emsp; 就是`dataloader`模块的职责，这也是本篇教程`tutorial-3`讲解的重点

**`dataloader`模块的职责**，详细划分可以包含以下三部分，**采样划分、补零对齐、框架匹配**

&emsp; &emsp; 第一，确定`batch`大小，确定采样方式，划分后通过迭代器即可得到`batch`序列

&emsp; &emsp; 第二，对于序列处理，这也是`fastNLP`主要针对的，将同个`batch`内的数据对齐

&emsp; &emsp; 第三，**`batch`内数据格式要匹配框架**，**但`batch`结构需保持一致**，**参数匹配机制**

&emsp; 对此，`fastNLP 0.8`给出了 **`TorchDataLoader`、`PaddleDataLoader`和`JittorDataLoader`**

&emsp; &emsp; 分别针对并匹配不同框架，但彼此之间参数名、属性、方法仍然类似，前两者大致如下表所示

| <div align="center">名称</div> | <div align="center">参数</div> | <div align="center">属性</div> | <div align="center">功能</div> | <div align="center">内容</div> |
|:--|:--:|:--:|:--|:--|
| **`dataset`** | √ | √ | 指定`dataloader`的数据内容  |  |
| `batch_size` | √ | √ | 指定`dataloader`的`batch`大小 | 默认`16` |
| `shuffle` | √ | √ | 指定`dataloader`的数据是否打乱 | 默认`False` |
| `collate_fn` | √ | √ | 指定`dataloader`的`batch`打包方法 | 视框架而定 |
| `sampler` | √ | √ | ？ | 默认`None` |
| `batch_sampler` | √ | √ | ？ | 默认`None` |
| `drop_last` | √ | √ | 指定`dataloader`划分`batch`时是否丢弃剩余的 | 默认`False` |
| `cur_batch_indices` |  | √ | 记录`dataloader`当前遍历批量序号 |  |
| `num_workers` | √ | √ | 指定`dataloader`开启子进程数量 | 默认`0` |
| `worker_init_fn` | √ | √ | 指定`dataloader`子进程初始方法 | 默认`None` |
| `generator` | √ | √ | 指定`dataloader`子进程随机种子 | 默认`None` |
| `prefetch_factor` |  | √ | 指定为每个`worker`装载的`sampler`数量 | 默认`2` |

&emsp; 论及`dataloader`的函数，其中，`get_batch_indices`用来获取当前遍历到的`batch`序号，其他函数

&emsp; &emsp; 包括`set_ignore`、`set_pad`和`databundle`类似，请参考`tutorial-2`，此处不做更多介绍

&emsp; &emsp; 以下是`tutorial-2`中已经介绍过的数据预处理流程，接下来是对相关数据进行`dataloader`处理

In [5]:
import sys
sys.path.append('..')

import pandas as pd
from functools import partial
from fastNLP.transformers.torch import BertTokenizer

from fastNLP import DataSet
from fastNLP import Vocabulary
from fastNLP.io import DataBundle


class PipeDemo:
    def __init__(self, tokenizer='bert-base-uncased'):
        self.tokenizer = BertTokenizer.from_pretrained(tokenizer)

    def process_from_file(self, path='./data/test4dataset.tsv'):
        datasets = DataSet.from_pandas(pd.read_csv(path, sep='\t'))
        train_ds, test_ds = datasets.split(ratio=0.7)
        train_ds, dev_ds = datasets.split(ratio=0.8)
        data_bundle = DataBundle(datasets={'train': train_ds, 'dev': dev_ds, 'test': test_ds})

        encode = partial(self.tokenizer.encode_plus, max_length=100, truncation=True,
                         return_attention_mask=True)
        data_bundle.apply_field_more(encode, field_name='Sentence', progress_bar='tqdm')
        
        target_vocab = Vocabulary(padding=None, unknown=None)

        target_vocab.from_dataset(*[ds for _, ds in data_bundle.iter_datasets()], field_name='Sentiment')
        target_vocab.index_dataset(*[ds for _, ds in data_bundle.iter_datasets()], field_name='Sentiment',
                                   new_field_name='target')

        data_bundle.set_pad('input_ids', pad_val=self.tokenizer.pad_token_id)
        data_bundle.set_ignore('SentenceId', 'Sentence', 'Sentiment')  
        return data_bundle

    
pipe = PipeDemo(tokenizer='bert-base-uncased')

data_bundle = pipe.process_from_file('./data/test4dataset.tsv')

Processing:   0%|          | 0/4 [00:00<?, ?it/s]

Processing:   0%|          | 0/2 [00:00<?, ?it/s]

Processing:   0%|          | 0/2 [00:00<?, ?it/s]

+------------+------------------+-----------+------------------+--------------------+--------------------+
| SentenceId | Sentence         | Sentiment | input_ids        | token_type_ids     | attention_mask     |
+------------+------------------+-----------+------------------+--------------------+--------------------+
| 5          | A comedy-dram... | positive  | [101, 1037, 4... | [0, 0, 0, 0, 0,... | [1, 1, 1, 1, 1,... |
| 2          | This quiet , ... | positive  | [101, 2023, 4... | [0, 0, 0, 0, 0,... | [1, 1, 1, 1, 1,... |
| 1          | A series of e... | negative  | [101, 1037, 2... | [0, 0, 0, 0, 0,... | [1, 1, 1, 1, 1,... |
| 6          | The Importanc... | neutral   | [101, 1996, 5... | [0, 0, 0, 0, 0,... | [1, 1, 1, 1, 1,... |
+------------+------------------+-----------+------------------+--------------------+--------------------+


### 1.2 dataloader 的函数创建

在`fastNLP 0.8`中，**更方便、可能更常用的`dataloader`创建方法是通过`prepare_xx_dataloader`函数**

&emsp; 例如下方的`prepare_torch_dataloader`函数，指定必要参数，读取数据集，生成对应`dataloader`

&emsp; 类型为`TorchDataLoader`，只能适用于`pytorch`框架，因此对应`trainer`初始化时`driver='torch'`

In [7]:
from fastNLP import prepare_torch_dataloader

train_dataset = data_bundle.get_dataset('train')
evaluate_dataset = data_bundle.get_dataset('dev')

train_dataloader = prepare_torch_dataloader(train_dataset, batch_size=16, shuffle=True)
evaluate_dataloader = prepare_torch_dataloader(evaluate_dataset, batch_size=16)

```python
trainer = Trainer(
    model=model,
    train_dataloader=train_dataloader,
    optimizers=optimizer,
	...
	driver='torch',
	device='cuda',
	...
    evaluate_dataloaders=evaluate_dataloader,     
    metrics={'acc': Accuracy()},
	...
)
```

之所以称`prepare_xx_dataloader`函数更方便，是因为其**导入对象不仅可也是`DataSet`类型**，**还可以**

&emsp; **是`DataBundle`类型**，不过数据集名称需要是`'train'`、`'dev'`、`'test'`供`fastNLP`识别

&emsp; 例如下方就是**直接通过`prepare_paddle_dataloader`函数生成基于`PaddleDataLoader`的字典**

&emsp; 在接下来`trainer`的初始化过程中，按如下方式使用即可，除了初始化时`driver='paddle'`外

&emsp; &emsp; 这里也可以看出 **`evaluate_dataloaders`的妙处**，一次评测可以针对多个数据集

In [6]:
from fastNLP import prepare_paddle_dataloader

dl_bundle = prepare_paddle_dataloader(data_bundle, batch_size=16, shuffle=True)

```python
trainer = Trainer(
    model=model,
    train_dataloader=dl_bundle['train'],
    optimizers=optimizer,
	...
	driver='paddle',
	device='gpu',
	...
    evaluate_dataloaders={'dev': dl_bundle['dev'], 'test': dl_bundle['test']},     
    metrics={'acc': Accuracy()},
	...
)
```

## 2. fastNLP 中 dataloader 的延伸

### 2.1 collator 的概念与使用

在`fastNLP 0.8`中，在数据加载模块`DataLoader`之前，还存在其他的一些模块，负责例如对文本数据

&emsp; 进行补零对齐，即 **核对器`collator`模块**，进行分词标注，即 **分词器`tokenizer`模块**

&emsp; 本节将对`fastNLP`中的核对器`collator`等展开介绍，分词器`tokenizer`将在下一节中详细介绍

在`fastNLP 0.8`中，**核对器`collator`模块负责文本序列的补零对齐**，通过

In [ ]:
from fastNLP import prepare_torch_dataloader

dl_bundle = prepare_torch_dataloader(data_bundle, train_batch_size=2)

print(type(dl_bundle), type(dl_bundle['train']))

&emsp; 

In [ ]:
dataloader = prepare_torch_dataloader(datasets['train'], train_batch_size=2)
print(type(dataloader))
print(dir(dataloader))

In [ ]:
dataloader.collate_fn

### 2.2 sampler 的概念与使用

In [ ]:
dataloader.batch_sampler

&emsp; 